<table align="left">
  <tr>
    <td><img src="fleet.png" alt="fleet of icecream trucks" width="120"/></td>
    <td align="left"><h1>Lesson 4: Monitoring and Evaluating your Agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Setup Tracing

In [16]:
PROJECT_NAME = "Customer-Success"

In [17]:
import os
from dotenv import load_dotenv, find_dotenv
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

tracer_provider = register(
    project_name=PROJECT_NAME,
    #endpoint= get_phoenix_endpoint() + "v1/traces"
    endpoint = os.getenv('DLAI_LOCAL_URL').format(port='6006') + "v1/traces"
)
SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Customer-Success
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://s172-29-11-220p6006.lab-aws-production.deeplearning.ai/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [18]:
from dotenv import load_dotenv, find_dotenv
load_dotenv() # load variables from local .env file

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

In [19]:
from smolagents import HfApiModel

model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

model([{"role": "user", "content": "What is your name?!"}])

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='My name is Qwen.', tool_calls=[], raw=ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='My name is Qwen.', tool_call_id=None, tool_calls=[]), logprobs=None, seed=18033466357982405000)], created=1747038476, id='ntior4S-4yUbBN-93e88a2b69f5b842', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=ChatCompletionOutputUsage(completion_tokens=7, prompt_tokens=34, total_tokens=41), object='chat.completion', prompt=[]))

In [20]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-11-220p6006.lab-aws-production.deeplearning.ai/


## Trace an agent run

In [21]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(model=model, tools=[])

>Note, the following line will sometimes get a timeout on the interface to the tracing package due to the networked interface. If this happens, try it again.


In [22]:
agent.run("What is the 10th Fibonacci number?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 10th Fibonacci number?                                                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_10 = fibonacci(10)                                                                                           
  print(fib_10)                                                                                                    
                                                                                                                   
  final_answer(fib_10)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
55

Out - Final answer: 55

[Step 1: Duration 0.11 seconds| Input tokens: 2,019 | Output tokens: 182]

55

In [23]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-11-220p6006.lab-aws-production.deeplearning.ai/


## Setup ice cream production system

In [24]:
from smolagents import tool
from typing import Dict

menu_prices = {"crepe nutella": 1.50, "vanilla ice cream": 2, "maple pancake": 1.}

ORDER_BOOK = {}

@tool
def place_order(quantities: Dict[str, int], session_id: int) -> None:
    """Places a pre-order of snacks.

    Args:
        quantities: a dictionary with names as keys and quantities as values
        session_id: the id for the client session
    """
    global ORDER_BOOK
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    ORDER_BOOK[session_id] = quantities

@tool
def get_prices(quantities: Dict[str, int]) -> str:
    """Gets price for certain quantities of ice cream.

    Args:
        quantities: a dictionary with names as keys and quantities as values
    """
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    total_price = sum([menu_prices[key] * value for key, value in quantities.items()])
    return (
        f"Given the current menu prices:\n{menu_prices}\nThe total price for your order would be: ${total_price}"
    )

In [25]:
order_agent = CodeAgent(
    tools=[place_order, get_prices],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
)

In [26]:
order_agent.run(
    "Could I come and collect one crepe nutella?",
    additional_args={"session_id": 192}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 192}.                                                                                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  place_order(quantities={'crepe nutella': 1}, session_id=session_id)                                              
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.11 seconds| Input tokens: 2,193 | Output tokens: 68]

'Order placed for one crepe nutella.'

### Try multiple orders

In [27]:
client_requests = [
    ("Could I come and collect one crepe nutella?", "place_order"),
    ("What would be the price for 1 crêpe nutella + 2 pancakes?", "get_prices"),
    ("How did you start your ice-cream business?", None),
    ("What's the weather at the Louvre right now?", None),
    ("I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want it. If it's below, I'll order it, please.", "place_order")
]

In [28]:
for request in client_requests:
    order_agent.run(
        request[0],
        additional_args={"session_id": 0, "menu_prices": menu_prices}
    )

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  quantities = {'crepe nutella': 1}                                                                                
  place_order(quantities=quantities, session_id=session_id)                                                        
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.11 seconds| Input tokens: 2,227 | Output tokens: 93]

...

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want    │
│ it. If it's below, I'll order it, please.                                                                       │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  vanilla_ice_cream_price = menu_prices['vanilla ice cream']                                                       
  print(f"The price of vanilla ice cream is ${vanilla_ice_cream_price}.")                                          
                                                                                                                   
  final_answer("I will not order the vanilla ice cream as it is more than $1.")                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The price of vanilla ice cream is $2.

Out - Final answer: I will not order the vanilla ice cream as it is more than $1.

[Step 1: Duration 0.08 seconds| Input tokens: 2,260 | Output tokens: 162]

In [29]:
import phoenix as px

spans = px.Client().get_spans_dataframe(project_name=PROJECT_NAME)
spans.head(20)

/usr/local/lib/python3.11/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (9.0.1) and client (8.19.1) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.input.mime_type,attributes.llm.output_messages,attributes.llm.token_count.completion,attributes.llm.invocation_parameters,attributes.openinference.span.kind,attributes.llm.input_messages,attributes.tool.description,attributes.tool.parameters,attributes.tool.name,attributes.smolagents
context.span_id,,,,,,,,,,,,,,,,,,,,,
399306685e783f95,HfApiModel.__call__,LLM,None,2025-05-12 08:25:46.208361+00:00,2025-05-12 08:25:46.372174+00:00,OK,,[],399306685e783f95,7cc684e4d33ff01a5c3bad785a395853,...,application/json,"[{'message.role': 'assistant', 'message.conten...",10.0,{},LLM,None,None,None,None,None
c4f388c51d074531,HfApiModel.__call__,LLM,None,2025-05-12 08:25:58.553833+00:00,2025-05-12 08:25:59.434929+00:00,OK,,[],c4f388c51d074531,dc63bbdd59e0e6d6cb5045903841abee,...,application/json,"[{'message.role': 'assistant', 'message.conten...",33.0,{},LLM,None,None,None,None,None
1872d2d3d5fe3c2b,HfApiModel.__call__,LLM,None,2025-05-12 08:26:56.250125+00:00,2025-05-12 08:26:58.171666+00:00,OK,,[],1872d2d3d5fe3c2b,a337c1787e909c9649b2447b0edda2d3,...,application/json,"[{'message.role': 'assistant', 'message.conten...",52.0,{},LLM,None,None,None,None,None
5a678f68c949bc42,HfApiModel.__call__,LLM,None,2025-05-12 08:27:21.883438+00:00,2025-05-12 08:27:22.876591+00:00,OK,,[],5a678f68c949bc42,8e62f12861dd9f8830b9e50bcc453136,...,application/json,"[{'message.role': 'assistant', 'message.conten...",53.0,{},LLM,None,None,None,None,None
b9c80c5dda4c18a8,HfApiModel.__call__,LLM,None,2025-05-12 08:27:55.860661+00:00,2025-05-12 08:27:56.678696+00:00,OK,,[],b9c80c5dda4c18a8,c9c34955ed8a4cfeddfca665b7e7ec5c,...,application/json,"[{'message.role': 'assistant', 'message.conten...",7.0,{},LLM,None,None,None,None,None
4ecd437402c9822a,HfApiModel.__call__,LLM,73f9e50c35b03a76,2025-05-12 08:29:05.665631+00:00,2025-05-12 08:29:08.241112+00:00,OK,,[],4ecd437402c9822a,5da3a437c3e38d307249804f726def87,...,application/json,"[{'message.role': 'assistant', 'message.conten...",182.0,{},LLM,"[{'message.role': 'system', 'message.content':...",None,None,None,None
b2b4281d8e7efd80,FinalAnswerTool,TOOL,73f9e50c35b03a76,2025-05-12 08:29:08.309279+00:00,2025-05-12 08:29:08.309343+00:00,OK,,[],b2b4281d8e7efd80,5da3a437c3e38d307249804f726def87,...,None,None,NaN,None,TOOL,None,Provides a final answer to the given problem.,"{'answer': {'type': 'any', 'description': 'The...",final_answer,None
73f9e50c35b03a76,Step 1,CHAIN,22de848e966242f5,2025-05-12 08:29:05.665405+00:00,2025-05-12 08:29:08.331678+00:00,OK,,[],73f9e50c35b03a76,5da3a437c3e38d307249804f726def87,...,None,None,NaN,None,CHAIN,None,None,None,None,None
22de848e966242f5,CodeAgent.run,AGENT,None,2025-05-12 08:29:05.657122+00:00,2025-05-12 08:29:08.350403+00:00,OK,,[],22de848e966242f5,5da3a437c3e38d307249804f726def87,...,None,None,182.0,None,AGENT,None,None,None,None,"{'tools_names': ['final_answer'], 'max_steps':..."


### Add processing to extract desired information

In [30]:
import pandas as pd
import json

agents = spans[spans['span_kind'] == 'AGENT'].copy()
agents['task'] = agents['attributes.input.value'].apply(
    lambda x: json.loads(x).get('task') if isinstance(x, str) else None
)

tools = spans.loc[
    spans['span_kind'] == 'TOOL',
    ["attributes.tool.name", "attributes.input.value", "context.trace_id"]
].copy()

tools_per_task = agents[
    ["name", "start_time", "task", "context.trace_id"]
].merge(
    tools,
    on="context.trace_id",
    how="left",
)
tools_per_task.head()

,name,start_time,task,context.trace_id,attributes.tool.name,attributes.input.value
0,CodeAgent.run,2025-05-12 08:29:05.657122+00:00,What is the 10th Fibonacci number?,5da3a437c3e38d307249804f726def87,final_answer,"{""args"": [55], ""sanitize_inputs_outputs"": fals..."
1,CodeAgent.run,2025-05-12 08:44:01.511033+00:00,Could I come and collect one crepe nutella?,5d4dcff8d23e75e319f00147c1313c50,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
2,CodeAgent.run,2025-05-12 08:44:01.511033+00:00,Could I come and collect one crepe nutella?,5d4dcff8d23e75e319f00147c1313c50,final_answer,"{""args"": [""Order placed for one crepe nutella...."
3,CodeAgent.run,2025-05-12 08:45:04.198221+00:00,What is the 10th Fibonacci number?,f87d74ceab110b5d0ef98c24c86f25ad,final_answer,"{""args"": [55], ""sanitize_inputs_outputs"": fals..."
4,CodeAgent.run,2025-05-12 08:45:26.296033+00:00,Could I come and collect one crepe nutella?,3e99235be3e2916ba308d539150e4232,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."


### Now, compare tool calls with exected tool calls

In [31]:
def score_request(expected_tool: str, tool_calls: list):
    if expected_tool is None:
        return tool_calls == set(["final_answer"])
    else:
        return expected_tool in tool_calls

results = []
for request, expected_tool in client_requests:
    tool_calls = set(tools_per_task.loc[tools_per_task["task"] == request, "attributes.tool.name"].tolist())
    results.append(
        {
            "request": request,
            "tool_calls_performed": tool_calls,
            "is_correct": score_request(expected_tool, tool_calls)
        }
    )
pd.DataFrame(results)

,request,tool_calls_performed,is_correct
0,Could I come and collect one crepe nutella?,"{final_answer, place_order}",True
1,What would be the price for 1 crêpe nutella + ...,"{final_answer, get_prices}",True
2,How did you start your ice-cream business?,"{final_answer, get_prices}",False
3,What's the weather at the Louvre right now?,{final_answer},True
4,I'm not sure if I should order. I want a vanil...,{final_answer},False
